# Download Sentinel-1 RTC for each ASO raster
Given an ASO raster, find and download 1) the most proximate in time S1 RTC product 2) a "snow-off" RTC product from the preceeding summer. 

In [1]:
# based on exmaples from
# https://planetarycomputer.microsoft.com/docs/tutorials/cloudless-mosaic-sentinel2/
# https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a#Example-Notebook
from pystac.extensions.eo import EOExtension as eo
import pystac_client
import planetary_computer
import glob
import rioxarray as rxr
import re
import datetime
import pandas as pd
from shapely.geometry import box
import odc.stac
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

In [2]:
def rtc_for_aso_snowon(aso_raster_fn):
    
    time = pd.to_datetime(re.search("(\d{4}\d{2}\d{2})", aso_raster_fn).group())
    week_before = (time - datetime.timedelta(weeks=1)).strftime('%Y-%m-%d')
    week_after = (time + datetime.timedelta(weeks=1)).strftime('%Y-%m-%d')
    time_of_interest = f'{week_before}/{week_after}'
    
    aso_raster = rxr.open_rasterio(aso_raster_fn).squeeze()
    aso_raster = aso_raster.where(aso_raster>=0, drop=True)
    aso_raster = aso_raster.interpolate_na(dim='x')
    bounds_latlon = box(*aso_raster.rio.transform_bounds("EPSG:4326"))
    
    catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace)

    search = catalog.search(
        collections=["sentinel-1-rtc"],
        intersects=bounds_latlon,
        datetime=time_of_interest)

    # Check how many items were returned
    items = search.item_collection()
    print(f"Returned {len(items)} Items")
    
    rtc_stac = odc.stac.load(items,chunks={"x": 2048, "y": 2048},resolution=50, groupby='sat:absolute_orbit')
    rtc_stac_clipped = rtc_stac.rio.clip_box(*bounds_latlon.bounds,crs="EPSG:4326")
    
    rel_orbits = [scene.properties['sat:relative_orbit'] for scene in items.items]
    ac_times = [scene.properties['datetime'] for scene in items.items]
    ac_times = [np.datetime64(item) for item in ac_times]
    
    # clip to ASO extent
    rtc_stac_clipped = rtc_stac_clipped.rio.reproject_match(aso_raster).where(aso_raster>=0, drop=True)
    
    # limit to morning acquisitions
    rtc_ds = rtc_stac_clipped.where(rtc_stac_clipped.time.dt.hour > 11, drop=True)
    if rtc_ds.vv.size == 0 :
        print('No morning acquisitions found')
        return None
    
    # calculate percent vh coverage of each acquisition
    perc_cover = (rtc_ds.vh > 0).sum(dim=['x', 'y'])/(rtc_ds.vh >= -1000000000).sum(dim=['x', 'y'])
    
    # if multiple with full coverage, grab nearest in time with full coverage
    if perc_cover.values.tolist().count(1) > 1:
        rtc_ds = rtc_ds.where(perc_cover == 1, drop=True).sortby('time')
        rtc_ds = rtc_ds.sel(time=time, method='nearest')
    
    # exit if no rasters have good vh coverage
    elif perc_cover.max() < 0.1:
        print('max vh coverage is < 10 percent--recommend skipping ASO raster')
        return None
     
    # otherwise, grab max coverage 
    else:
        rtc_ds = rtc_ds.sel(time=perc_cover.idxmax())
    
    # mask negative areas
    rtc_ds = rtc_ds.where(rtc_ds.vh > 0, drop=True)
    
    # get relative orbit of scene
    rel_orbit = rel_orbits[ac_times.index(rtc_ds.time)]
    
    #rtc_ds.to_netcdf(f'../data/S1_rtc/S1_snow-on_{rtc_ds.time.dt.strftime("%Y%m%d").item()}_for_{aso_raster_fn[8:-4]}.nc')
    
    return rel_orbit

In [3]:
# fn = '/home/jovyan/crunchy-snow/data/ASO/ASO_50m_SD_cleaned/utm12n/ASO_50M_SD_WindRiver_20220611_clean.tif'

In [4]:
# rel_orbit, rtc_ds, aso = rtc_for_aso_snowon(fn)

In [5]:
# f, ax = plt.subplots(1, 2, figsize = (10, 10))
# ax[0].imshow(rtc_ds.vh, vmax=0.1, vmin=0)
# ax[0].set_aspect('equal')
# ax[1].imshow(aso, vmin=0, vmax=2, cmap = "Blues")
# ax[1].set_aspect('equal')
# plt.tight_layout()

In [6]:
def rtc_for_aso_snowoff(aso_raster_fn, orbit):
    
    if orbit == None:
        print('no orbit supplied, skipping')
        return None
    
    year = pd.to_datetime(re.search("(\d{4}\d{2}\d{2})", aso_raster_fn).group()).year
    time = pd.to_datetime(f'{year-1}0910')
    week_before = (time - datetime.timedelta(weeks=1)).strftime('%Y-%m-%d')
    week_after = (time + datetime.timedelta(weeks=1)).strftime('%Y-%m-%d')
    time_of_interest = f'{week_before}/{week_after}'
    
    aso_raster = rxr.open_rasterio(aso_raster_fn).squeeze()
    aso_raster = aso_raster.where(aso_raster>=0, drop=True)
    aso_raster = aso_raster.interpolate_na(dim='x')
    bounds_latlon = box(*aso_raster.rio.transform_bounds("EPSG:4326"))
    
    catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace)

    search = catalog.search(
        collections=["sentinel-1-rtc"],
        intersects=bounds_latlon,
        datetime=time_of_interest)

    # Check how many items were returned
    items = search.item_collection()
    print(f"Returned {len(items)} Items")
    
    # grab only items with appropriate relative orbit
    item_list = []
    for item in items:
        if item.properties["sat:relative_orbit"] == orbit:
            item_list.append(items)
    
    rtc_stac = odc.stac.load(items,chunks={"x": 2048, "y": 2048},resolution=50, groupby='sat:absolute_orbit')
    rtc_stac_clipped = rtc_stac.rio.clip_box(*bounds_latlon.bounds,crs="EPSG:4326")
    
    # clip to ASO extent
    rtc_stac_clipped = rtc_stac_clipped.rio.reproject_match(aso_raster).where(aso_raster>=0, drop=True)
    
    # limit to morning acquisitions
    rtc_ds = rtc_stac_clipped.where(rtc_stac_clipped.time.dt.hour > 11, drop=True)
    
    # calculate percent vh coverage of each acquisition
    perc_cover = (rtc_ds.vh > 0).sum(dim=['x', 'y'])/(rtc_ds.vh >= -1000000000).sum(dim=['x', 'y'])
    
    # if multiple with full coverage, grab nearest in time with full coverage
    if perc_cover.values.tolist().count(1) > 1:
        rtc_ds = rtc_ds.where(perc_cover == 1, drop=True).sortby('time')
        rtc_ds = rtc_ds.sel(time=time, method='nearest')
    
    # exit if no rasters have good vh coverage
    elif perc_cover.max() < 0.1:
        print('max vh coverage is < 10 percent--recommend skipping ASO raster')
        return None
     
    # otherwise, grab max coverage 
    else:
        rtc_ds = rtc_ds.sel(time=perc_cover.idxmax())
    
    # mask negative areas
    rtc_ds = rtc_ds.where(rtc_ds.vh > 0, drop=True)
    
    #rtc_ds.to_netcdf(f'../data/S1_rtc/S1_snow-off_{rtc_ds.time.dt.strftime("%Y%m%d").item()}_for_{aso_raster_fn[8:-4]}.nc')

In [7]:
# rtc_ds, aso = rtc_for_aso_snowoff(fn, rel_orbit)

In [8]:
# rtc_ds

In [9]:
# f, ax = plt.subplots(1, 2, figsize = (10, 10), sharex=True, sharey=True)
# ax[0].imshow(rtc_ds.vh, vmax=0.1, vmin=0)
# ax[0].set_aspect('equal')
# ax[1].imshow(aso, vmin=0, vmax=2, cmap = "Blues")
# ax[1].set_aspect('equal')
# plt.tight_layout()

In [10]:
def rtc_for_aso_all(dir_path):
    raster_paths = glob.glob(f'{dir_path}/*/ASO_50M_SD*.tif')
    for i, path in enumerate(raster_paths):
        print(f'----\nworking on {path.split("/")[-1]}, {i+1}/{len(raster_paths)}\n----')
        
        relative_orbit = rtc_for_aso_snowon(path)
        rtc_for_aso_snowoff(path, relative_orbit)

In [ ]:
dir_path = '/home/jovyan/crunchy-snow/data/ASO/ASO_50m_SD_cleaned'
test = rtc_for_aso_all(dir_path)

----
working on ASO_50M_SD_BlueRiver_20230529_clean.tif, 1/243
----
Returned 2 Items


/tmp/ipykernel_3034/2749076730.py:31: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  ac_times = [np.datetime64(item) for item in ac_times]
/srv/conda/envs/notebook/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


No morning acquisitions found
no orbit supplied, skipping
----
working on ASO_50M_SD_Animas_20210419_clean.tif, 2/243
----
Returned 6 Items
Returned 6 Items
----
working on ASO_50M_SD_EastRiver_20230523_clean.tif, 3/243
----
Returned 5 Items
Returned 4 Items
----
working on ASO_50M_SD_EastRiver_20220518_clean.tif, 4/243
----
Returned 3 Items
Returned 6 Items
----
working on ASO_50M_SD_SouthPlatte_20230416_clean.tif, 5/243
----
Returned 6 Items
Returned 8 Items
----
working on ASO_50M_SD_USCOCM_20190407_clean.tif, 6/243
----
Returned 3 Items
Returned 4 Items
----
working on ASO_50M_SD_TaylorAndLottis_20230523_clean.tif, 7/243
----
Returned 4 Items
Returned 3 Items
----
working on ASO_50M_SD_USCOGE_20180331_clean.tif, 8/243
----
Returned 5 Items
Returned 8 Items
